In [1]:
from src.lib.MONGODB import *
from src.system.secret import *
import pandas as pd
from src.modules.ALGORITHM_MODULE import *
import numpy as np

conn = MongoDbFunctions(
    DATABASE["host"],
    DATABASE["port"],
    DATABASE["username"],
    DATABASE["password"],
    DATABASE["dbname"],
    "CoreData",
)

assets = conn.findByMultipleFields(
    fields={"interval": "15min", "symbol": {"$ne": "SPX"}},
    custom=True,
    get_all=True,
    proyeccion={"_id": 0, "symbol": 1},
)
assets = [asset["symbol"] for asset in assets]


## Obtenemos AMD

In [2]:

data = conn.findByMultipleFields(
    fields={"symbol": "AAPL", "interval": "15min"}, custom=True
)

data = parameters.formatData(data)

print(f"Last data date: {data.iloc[0]['date']}")

Last data date: 2021-01-04 04:00:00


## Calculamos simple moving average

In [3]:
parameters.simplemovingaverage(data, 50)
parameters.simplemovingaverage(data, 200)
print(data)

                        open     high      low    close   volume  \
datetime                                                           
2021-01-04 04:00:00  131.001  131.698  130.688  131.423  40911.0   
2021-01-04 04:15:00  131.659  131.708  131.484  131.678  26646.0   
2021-01-04 04:30:00  131.679  131.698  131.209  131.531   6493.0   
2021-01-04 04:45:00  131.541  131.560  131.160  131.285  13911.0   
2021-01-04 05:00:00  131.286  131.649  131.258  131.285  31283.0   
...                      ...      ...      ...      ...      ...   
2023-12-01 18:45:00  191.275  191.320  191.210  191.210   3017.0   
2023-12-01 19:00:00  191.240  191.300  191.220  191.280   1383.0   
2023-12-01 19:15:00  191.290  191.300  191.260  191.280   2948.0   
2023-12-01 19:30:00  191.280  191.320  191.260  191.280    985.0   
2023-12-01 19:45:00  191.290  191.330  191.270  191.300   6707.0   

                                   date    SMA_50    SMA_200  
datetime                                            

## Calculo de relative strength index

In [4]:
parameters.relativestregthindex(data, 14)
print(data)

                        open     high      low    close   volume  \
datetime                                                           
2021-01-04 04:00:00  131.001  131.698  130.688  131.423  40911.0   
2021-01-04 04:15:00  131.659  131.708  131.484  131.678  26646.0   
2021-01-04 04:30:00  131.679  131.698  131.209  131.531   6493.0   
2021-01-04 04:45:00  131.541  131.560  131.160  131.285  13911.0   
2021-01-04 05:00:00  131.286  131.649  131.258  131.285  31283.0   
...                      ...      ...      ...      ...      ...   
2023-12-01 18:45:00  191.275  191.320  191.210  191.210   3017.0   
2023-12-01 19:00:00  191.240  191.300  191.220  191.280   1383.0   
2023-12-01 19:15:00  191.290  191.300  191.260  191.280   2948.0   
2023-12-01 19:30:00  191.280  191.320  191.260  191.280    985.0   
2023-12-01 19:45:00  191.290  191.330  191.270  191.300   6707.0   

                                   date    SMA_50    SMA_200     RSI_14  
datetime                                 

## Calculo de Beta diaria (Últimos 5 años)

In [5]:
year = data.iloc[0]["date"].year
betas = parameters.beta("AMD",limit=str(year))
print(betas)

              open      high      low   close      volume       date      beta
datetime                                                                      
2021-01-04   92.11   96.0600   90.915   92.30  51802552.0 2021-01-04  1.487356
2021-01-05   92.10   93.2077   91.410   92.77  34207989.0 2021-01-05  1.481697
2021-01-06   91.62   92.2800   89.460   90.33  51911657.0 2021-01-06  1.472786
2021-01-07   91.33   95.5100   91.200   95.16  42897157.0 2021-01-07  1.472131
2021-01-08   95.98   96.4000   93.270   94.58  39816400.0 2021-01-08  1.471565
...            ...       ...      ...     ...         ...        ...       ...
2023-11-27  122.07  123.8900  121.772  122.65  32743006.0 2023-11-27  1.602713
2023-11-28  121.98  122.5400  120.790  122.01  32047063.0 2023-11-28  1.605033
2023-11-29  123.98  125.7250  123.480  123.85  37876317.0 2023-11-29  1.605013
2023-11-30  123.61  124.0500  119.650  121.16  44494211.0 2023-11-30  1.605275
2023-12-01  119.88  121.4001  118.645  121.39  37048

## Asignamos el valor de beta a cada día de la serie de 15 minutos

In [6]:
from src.modules.ALGORITHM_MODULE import *

parameters.apply_beta(data, betas)


In [7]:
BackUp = data.copy()

## Obtenemos el último la fecha de balance sheet

In [8]:
parameters.asignReportPresentationDate("AMD",data)
print(data)

                        open     high      low    close   volume  \
datetime                                                           
2021-01-04 04:00:00  131.001  131.698  130.688  131.423  40911.0   
2021-01-04 04:15:00  131.659  131.708  131.484  131.678  26646.0   
2021-01-04 04:30:00  131.679  131.698  131.209  131.531   6493.0   
2021-01-04 04:45:00  131.541  131.560  131.160  131.285  13911.0   
2021-01-04 05:00:00  131.286  131.649  131.258  131.285  31283.0   
...                      ...      ...      ...      ...      ...   
2023-12-01 18:45:00  191.275  191.320  191.210  191.210   3017.0   
2023-12-01 19:00:00  191.240  191.300  191.220  191.280   1383.0   
2023-12-01 19:15:00  191.290  191.300  191.260  191.280   2948.0   
2023-12-01 19:30:00  191.280  191.320  191.260  191.280    985.0   
2023-12-01 19:45:00  191.290  191.330  191.270  191.300   6707.0   

                                   date    SMA_50    SMA_200     RSI_14  \
datetime                                

In [9]:
BackUp = data.copy()

## Calculo de comportamiento en días de presentación de resultados


In [10]:
from src.modules.ALGORITHM_MODULE import *

parameters.asignMovement("AAPL",data)


## BackUp Físico


In [11]:
import pandas as pd

# Especifica la ruta donde quieres guardar el archivo
ruta_archivo = '/Users/oscarperezarruti/Documents/Documentos/Repositorios/trabajo-fin-grado-23/data_backup.xlsx'

# Guardar el DataFrame en un archivo de Excel
data.to_excel(ruta_archivo, index=False, engine='openpyxl')